In [22]:
import os

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [23]:
from pydantic import BaseModel
from typing import List

class filePath(BaseModel):
    mainFile: str
    relatedFiles: List[str]
    imageFiles: List[str]

In [24]:
from crewai import Agent
from crewai_tools import FileReadTool, DirectoryReadTool, VisionTool


class Agents:
    def markdownPathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the markdown files inside {file_path} path",
            backstory="You are fluent in Korean, and you are very good at finding markdown files.",
            allow_delegation=False,
            verbose=True,
            tools=[
                DirectoryReadTool(),
            ],
        )

    def imgPathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the img files inside {img_path} path",
            backstory="You are fluent in Korean, and you are very good at finding image files.",
            allow_delegation=False,
            verbose=True,
            tools=[
                DirectoryReadTool(),
            ],
        )

    def fileSelector(self):
        return Agent(
            role="fileSelector",
            goal="Search One file that fits {question}, and Make one file that should be collected the paths of all files associated with the file.",
            backstory="You are a file search expert and fluent in Korean. You have a great ability to read and analyze the details of the file.",
            allow_delegation=False,
            verbose=True,
            tools=[
                FileReadTool(),
            ],
        )

    def imgExtracter(self):
        return Agent(
            role="imgExtracter",
            goal="Extract the image files. and Make text file containing the contents of the image ",
            backstory="You are fluent in Korean, and You have a good ability to read images and convert them into text.",
            allow_delegation=False,
            verbose=True,
            tools=[
                VisionTool(),
            ],
        )

    def imgExtracter(self):
        return Agent(
            role="imgExtracter",
            goal="Extract the image files. and Make text file containing the contents of the image ",
            backstory="You are fluent in Korean, and You have a good ability to read images and convert them into text.",
            allow_delegation=False,
            verbose=True,
            tools=[
                VisionTool(),
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Finds and summarizes the markdown files that can solve the {question}",
            backstory="You are fluent in Korean, and you are very good at finding and summarizing files. Summarized information is essential for problem solving",
            verbose=True,
            tools=[
                FileReadTool(),
            ],
            max_iter=10,
        )

In [25]:
from crewai import Task


class Tasks:
    def markdownPathSearch(self, agent):
        return Task(
            description="Finds ALL the markdown files and inside {file_path} path",
            expected_output="Your final answer MUST be markdown file path",
            agent=agent,
            output_file="MarkdownPath.md",
        )

    def imgPathSearch(self, agent):
        return Task(
            description="Finds ALL the image files and inside {img_path} path. but NOT Include svg Image.",
            expected_output="Your final answer MUST be image path. svg images should NEVER be included.",
            agent=agent,
            output_file="ImgPath.md",
        )

    def fileSelect(self, agent, context):
        return Task(
            description="""
            Based on the markdownPathSearch and imgPathSearch, Search Only one file that can solve {question}. There are other documents linked by the symbol '[[...]]' and '![[...]]' in that file NOT '[...]'. 
            '[[...]]' symbol means a markdown file and '![[...]]' means an image file.
            
            Find all of the '[[...]]' and '![[...]]' and print out the ONLY file path associated with the word in it in markdownPathSearch or imgPathSearch. 
            All file paths should EXIST in that markdownPathSearch Output or imgPathSearch Output. 
            If the relevant document/image does not exist, JUST Return EMPTY List.",
            """,
            expected_output="""
            Your final answer MUST include the path of the first file and the path of other files within that file.
            It doesn't include ANYTHING other than file paths. 

            Example Answer 1
            {
                "mainFile": ".\\Algorithm\\Algorithm Content\\Tree\\MST(Minimum Spanning Tree).md",
                "relatedFiles": [
                    ".\\Algorithm\\Algorithm Content\\Graph Theory\\DFS(Depth-First Search).md",
                    ".\\Algorithm\\Algorithm Content\\Graph Theory\\BFS(Breadth-First Search).md",
                    ".\\Algorithm\\Algorithm Content\\Tree\\Union Find.md",
                ],
                "imageFiles": [".\\Algorithm\\Reference\\Tree Reference\\MST Ref\\MST Graph.png",]
            }

            Example Answer 2
            {
                "mainFile": ".\\Algorithm\\Algorithm Content\\Graph Theory\\BFS(Breadth-First Search).md",
                "relatedFiles": [],
                "imageFiles": [
                    ".\\Algorithm\\Reference\\Graph Theory Reference\\BASE TREE.png",
                    ".\\Algorithm\\Reference\\Graph Theory Reference\\BFS Ref\\BFS Queue.png",
                ]
            }

            Example Answer 3
            {
                "mainFile": ".\\c\\k.md",
                "relatedFiles": [
                    ".\\c\\g.md",
                    ".\\c\\c.md",
                    ".\\c\\d.md",
                ],
                "imageFiles": []
            }             
            """,
            agent=agent,
            context=context,
            output_json=filePath,
            output_file="associateFilePath.md",
        )

    def imgExtract(self, agent, content):
        return Task(
            description="Based on the fileSelect, Read all the img files in that # Image Files Part and Extract them in text.",
            expected_output="""
            Your final answer MUST be contents of the image in Korean. Don't summarize the contents of the image, JUST print it out as it is.
            """,
            agent=agent,
            content=content,
            output_file="ImgExtractContent.md",
        )

    def research(self, agent, context):
        return Task(
            description="Based on the pathSearch, Gather and analyze the most relevant markdown files for troubleshooting {question}. Provide a summary of markdown files and what is an important point in files. If there is a symbol '[[file_name]]' in the markdown file, you should also look into the markdown file with the file_name.",
            expected_output="Your final answer MUST be a detailed summary of the markdown files in Korean. Include Path to all imported files, why the file was imported, what is an important point and If there is an example for {question}, include it. If you can't find the file, JUST Answer that you don't know",
            agent=agent,
            context=context,
            output_file="questionAnswer.md",
        )

In [26]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agent = Agents()
tasks = Tasks()

markdownPathSearcher = agent.markdownPathSearcher()
imgPathSearcher = agent.imgPathSearcher()
fileSelector = agent.fileSelector()
imgExtracter = agent.imgExtracter()
researcher = agent.researcher()

markdownPathSearcher_task = tasks.markdownPathSearch(markdownPathSearcher)
imgPathSearcher_task = tasks.imgPathSearch(imgPathSearcher)
fileSelector_task = tasks.fileSelect(
    fileSelector,
    [markdownPathSearcher_task, imgPathSearcher_task],
)
imgExtracter_task = tasks.imgExtract(imgExtracter, [fileSelector_task])
researcher_task = tasks.research(researcher, [markdownPathSearcher_task])

crew = Crew(
    agents=[
        markdownPathSearcher,
        imgPathSearcher,
        fileSelector,
    ],
    tasks=[
        markdownPathSearcher_task,
        imgPathSearcher_task,
        fileSelector_task,
    ],
    memory=True,
    verbose=True,
)

result = crew.kickoff(
    dict(
        file_path=".\Algorithm\Algorithm Content",
        img_path=".\Algorithm\Reference",
        question="Fenwick 내용을 출력해줘",
    )
)

<>:40: SyntaxWarning: invalid escape sequence '\A'
<>:41: SyntaxWarning: invalid escape sequence '\A'
<>:40: SyntaxWarning: invalid escape sequence '\A'
<>:41: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Shind\AppData\Local\Temp\ipykernel_15356\464001493.py:40: SyntaxWarning: invalid escape sequence '\A'
  file_path=".\Algorithm\Algorithm Content",
C:\Users\Shind\AppData\Local\Temp\ipykernel_15356\464001493.py:41: SyntaxWarning: invalid escape sequence '\A'
  img_path=".\Algorithm\Reference",
C:\Users\Shind\AppData\Local\Temp\ipykernel_15356\464001493.py:40: SyntaxWarning: invalid escape sequence '\A'
  file_path=".\Algorithm\Algorithm Content",
C:\Users\Shind\AppData\Local\Temp\ipykernel_15356\464001493.py:41: SyntaxWarning: invalid escape sequence '\A'
  img_path=".\Algorithm\Reference",


ValidationError: 3 validation errors for filePath
mainFile
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
relatedFiles
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
imageFiles
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [56]:
fileSelector_task.output.raw

'{\n  "mainFile": "./Algorithm/Algorithm Content/Tree/Fenwick Tree.md",\n  "relatedFiles": [],\n  "imageFiles": [\n    "./Algorithm/Reference/Tree Reference/Fenwick Tree Ref/Fenwick Tree Partial Sum Graph.png",\n    "./Algorithm/Reference/Tree Reference/Fenwick Tree Ref/Fenwick Tree Range Update & Point Query Graph.png",\n    "./Algorithm/Reference/Tree Reference/Fenwick Tree Ref/Fenwick Tree Struct Graph.png"\n  ]\n}'